In [1]:
import numpy as np
import xarray as xr
import dask.array as da
import dask.dataframe as dd
import dask.bag as db


recordsarray = np.array([('Rex', 9, 81.0), ('Fido', 3, 27.0)], dtype=[('name', 'U10'), ('age', 'i4'), ('weight', 'f4')])
channelcoord = xr.DataArray(da.array(recordsarray), dims=["c"])
loaded = xr.DataArray(da.zeros((1024,1024,2,1)),dims=["x","y","c","z"], coords={"channels": channelcoord})

bag = db.from_sequence([loaded, loaded, loaded])

In [2]:
import django
django.setup()

In [10]:
from larvik.models import Zarr
from elements.models import Representation

In [14]:
nana = Zarr.objects.all()

In [15]:
nana

<ZarrQuerySet []>

In [13]:
nana.delete()

In [4]:
na = Representation.distributed.all().spread()

In [11]:
test = na.take(1)[0].compute()

In [13]:
test.plot.imshow()

ValueError: DataArray must be 2d

In [10]:
df.compute()

,Index,Name,EmissionWavelength,ExcitationWavelength,IlluminationType,AcquisitionMode,Color,SamplesPerPixel,Array
0,0,Ch1-T1,518.0,488.0,Epifluorescence,LaserScanningConfocalMicroscopy,16711935,1,Initial Stack
1,1,Ch1-T2,585.0,555.0,Epifluorescence,LaserScanningConfocalMicroscopy,-16776961,1,Initial Stack
0,0,Ch1-T1,518.0,488.0,Epifluorescence,LaserScanningConfocalMicroscopy,16711935,1,Max ISP of Initial Stack
1,1,Ch1-T2,585.0,555.0,Epifluorescence,LaserScanningConfocalMicroscopy,-16776961,1,Max ISP of Initial Stack
0,0,Ch1-T1,518.0,488.0,Epifluorescence,LaserScanningConfocalMicroscopy,16711935,1,Max ISP of Initial Stack
...,...,...,...,...,...,...,...,...,...
2,2,Alx647,825.0,NaN,None,None,65535,1,Max ISP of Initial Stack
0,0,Ch1,NaN,NaN,None,None,16711935,1,Max ISP of Initial Stack
1,1,Ch2,NaN,NaN,None,None,-16776961,1,Max ISP of Initial Stack
0,0,Ch1,NaN,NaN,None,None,16711935,1,Max ISP of Initial Stack


In [25]:
def get_biometa(array):
            df = dd.from_dask_array(array.channels.data)
            df["Array"] = array.name
            return df

newbag = bag.map(get_biometa)

In [26]:
%%time
newbag.take(1)[0].compute()

CPU times: user 44.8 ms, sys: 69.4 ms, total: 114 ms
Wall time: 167 ms


,name,age,weight,Array
0,Rex,9,81.0,zeros-7313d751e03555992b4a7de697c969b6
1,Fido,3,27.0,zeros-7313d751e03555992b4a7de697c969b6


In [15]:
class BioMetaAccesor(object):
    
    def __init__(self, bag):
        self.bag = bag
    
    @property
    def channels(self):
        
        def get_biometa(array):
            df = dd.from_dask_array(array.channels.data)
            df["Array"] = array.name
            return df
        
        datasets = self.bag.map(get_biometa)
        return datasets



class LarvikBag(object):

    def __init__(self, bag, *args, **kwargs):
        self.bag = bag
        self._biometa = None
        self.settings = {}

    def __getattr__(self, item):
        func = getattr(self.bag, item)
        def inner(*args, **kwargs):
            value = func(*args, **kwargs)
            if isinstance(value, Bag):
                return LarvikBag(value)
            else:
                return value

        return inner

    def parseSettings(self,settings):
        self.settings = settings

    def parse(self, *args):
        ''' :param *args Array of ParserSubclass'''
        parsedb = self.bag
        manager = LarvikManager()

        for i, parser in enumerate(args):
            parsedb = parsedb.map(parser.filter, self.settings, manager)

        return LarvikBag(parsedb)
    
    @property
    def biometa(self):
        if not self._biometa:
            self._biometa = BioMetaAccesor(self.bag)
        return self._biometa

    def __repr__(self):
        return self.bag.__repr__()

import dask.bag as db
bag =  LarvikBag(db.from_sequence([loaded, loaded, loaded]))

In [17]:
newbag = bag.map(get_biometa)

NameError: name 'Bag' is not defined